In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
import cv2
from keras.utils import to_categorical
from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
from keras.optimizers import SGD, Adam
from sklearn.model_selection import train_test_split


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Using TensorFlow backend.


In [2]:

img_height,img_width = 128,128 
num_classes = 10
#model articheture and added layers
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,1))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)
#the used optimizer
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
#reading and processing and normalizing training images
x_train = pd.read_pickle("/content/drive/My Drive/MNIST img recognition/modified-mnist/train_max_x")
x_train /= 255
x_train = x_train.reshape((-1, 128, 128,1))

In [0]:
#reading and processing and normalizing training labels
y_train  = pd.read_csv("/content/drive/My Drive/MNIST img recognition/modified-mnist/train_max_y.csv")
y_train = y_train["Label"].values
y_train = to_categorical(y_train)


In [0]:
#reading and processing and normalizing test images
x_test = pd.read_pickle("/content/drive/My Drive/MNIST img recognition/modified-mnist/test_max_x")
x_test /= 255

x_test = x_test.reshape((-1, 128, 128,1))
#splitting the train images and labels into validation images and labels
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)




In [6]:
mdfit = model.fit(x_train, y_train, epochs = 100, batch_size = 64, validation_data=(x_val, y_val))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 173s 4ms/step - loss: 2.7967 - acc: 0.2076 - val_loss: 2.1211 - val_acc: 0.2359
Epoch 2/100
40000/40000 [==============================] - 109s 3ms/step - loss: 2.1635 - acc: 0.2417 - val_loss: 3.1082 - val_acc: 0.2655
Epoch 3/100
40000/40000 [==============================] - 109s 3ms/step - loss: 2.0855 - acc: 0.2562 - val_loss: 2.8776 - val_acc: 0.2749
Epoch 4/100
40000/40000 [==============================] - 109s 3ms/step - loss: 2.0492 - acc: 0.2774 - val_loss: 2.2493 - val_acc: 0.2980
Epoch 5/100
40000/40000 [==============================] - 109s 3ms/step - loss: 2.0059 - acc: 0.3011 - val_loss: 3.2218 - val_acc: 0.2953
Epoch 6/100
40000/40000 [==============================] - 109s 3ms/step - loss: 1.9405 - acc: 0.3308 - val_loss: 1.7840 - val_acc: 0.3455
Epoch 7/100
40000/40

KeyboardInterrupt: ignored

In [0]:
preds = model.evaluate(x_val, y_val)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [0]:
result = model.predict(x_test, batch_size=512)
result = np.argmax(result, axis=1)



In [0]:
pd.DataFrame({'Id': np.arange(result.shape[0]), 'Label': result}).to_csv(r'/content/drive/My Drive/MNIST img recognition/output.csv', index=False)